In [3]:
import pandas as pd
import altair as alt

In [16]:
csv = pd.read_csv('./workers.csv')

In [17]:
df = pd.DataFrame(csv, columns=['date_of_employment'])
data = pd.DataFrame(df.value_counts(ascending=False).iloc[:25])
alt.Chart(data.reset_index().rename(columns={0: "counts"})).mark_bar(orient='vertical').encode(
    x='date_of_employment',
    y='counts', color='counts'
)

C:\Users\kaczm\PycharmProjects\staticticInPython\venv\lib\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [18]:
df = pd.DataFrame(csv, columns=['date_of_employment'])
df = df.groupby(df['date_of_employment'].map(lambda x: pd.Timestamp(x).year)).count()
df = df.rename(columns={'date_of_employment': 'counts'})
df["counts"] = df["counts"].astype('int32')
df = df.reset_index(level=0)
alt.Chart(df).mark_bar(orient='vertical').encode(
    alt.X('date_of_employment'), y='counts', color='counts'
)

C:\Users\kaczm\PycharmProjects\staticticInPython\venv\lib\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [19]:
df = pd.DataFrame(csv, columns=['id', 'employment_type'])
df = df.groupby(['employment_type']).count()
df = df.rename(columns={'id': 'counts'})
df["counts"] = df["counts"].astype('int32')
df = df.reset_index(level=0)
alt.Chart(df).mark_arc().encode(
    color='employment_type', theta='counts'
)

C:\Users\kaczm\PycharmProjects\staticticInPython\venv\lib\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [20]:
df = pd.DataFrame(csv, columns=['id', 'team'])
df = df.groupby(['team']).count()
df = df.rename(columns={'id': 'counts'})
df["counts"] = df["counts"].astype('int32')
df = df.reset_index(level=0)
alt.Chart(df).mark_bar(orient='horizontal').encode(
    x='counts', y='team', color='counts'
)

C:\Users\kaczm\PycharmProjects\staticticInPython\venv\lib\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [21]:
df = pd.DataFrame(csv, columns=['id', 'team', 'employment_type'])
df = df.groupby(['team', 'employment_type']).count()
df = df.rename(columns={'id': 'counts'})
df["counts"] = df["counts"].astype('int32')
df = df.reset_index(level=0)
df = df.reset_index(level=0)
alt.Chart(df).mark_bar(orient='horizontal').encode(
    column="team",
    x='employment_type', y='counts', color='counts'
)

C:\Users\kaczm\PycharmProjects\staticticInPython\venv\lib\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [22]:
df_elements = pd.DataFrame(pd.read_csv('widgets.csv'))


FileNotFoundError: [Errno 2] No such file or directory: 'elements.csv'

In [328]:
df = pd.DataFrame(pd.read_csv('employees.csv'))
df_elements = pd.DataFrame(pd.read_csv('elements.csv'))
df_merged = df.merge(df_elements, left_on='id', right_on='employee_id')
df_merged = df_merged.groupby(['team']).count()
df_merged = df_merged.rename(columns={'id': 'counts'})
df_merged["counts"] = df_merged["counts"].astype('int32')
df_merged = df_merged.reset_index(level=0)
alt.Chart(df_merged).mark_bar(orient='horizontal').encode(
    x='team', y='counts', color='counts'
)

/Users/pp/Library/Caches/pypoetry/virtualenvs/factorydb-fGh_wC37-py3.10/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [327]:
df = pd.DataFrame(pd.read_csv('employees.csv'))
df_elements = pd.DataFrame(pd.read_csv('elements.csv'))
df_merged = df.merge(df_elements, left_on='id', right_on='employee_id')
df_merged = df_merged.groupby(['employment_type']).count()
df_merged = df_merged.rename(columns={'id': 'counts'})
df_merged["counts"] = df_merged["counts"].astype('int32')
df_merged = df_merged.reset_index(level=0)
alt.Chart(df_merged).mark_bar(orient='horizontal').encode(
    x='employment_type', y='counts', color='counts'
)

/Users/pp/Library/Caches/pypoetry/virtualenvs/factorydb-fGh_wC37-py3.10/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [340]:
df = pd.DataFrame(pd.read_csv('employees.csv'))
df_elements = pd.DataFrame(pd.read_csv('elements.csv'))
df_merged = df.merge(df_elements, left_on='id', right_on='employee_id')
df_merged = df_merged.groupby(['name']).count()
df_merged = df_merged.reset_index(level=0)
df_merged = pd.DataFrame(df_merged, columns=['name', 'id'])
df_merged = df_merged.rename(columns={'id': 'counts'})
df_merged["counts"] = df_merged["counts"].astype('int32')
alt.Chart(df_merged.nsmallest(20, columns=['counts'])).mark_bar(orient='horizontal').encode(
    x='name', y='counts', color='counts'
)

/Users/pp/Library/Caches/pypoetry/virtualenvs/factorydb-fGh_wC37-py3.10/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)